In [3]:
import sqlite3
conn = sqlite3.connect('sqlite_db_pythonsqlite.db')
cur = conn.cursor()
cur.execute('select distinct name from facilities').fetchall()


[('Tennis Court 1',),
 ('Tennis Court 2',),
 ('Badminton Court',),
 ('Table Tennis',),
 ('Massage Room 1',),
 ('Massage Room 2',),
 ('Squash Court',),
 ('Snooker Table',),
 ('Pool Table',)]

In [5]:
''' Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members! '''

sql_query = '''
SELECT DISTINCT name AS facility_name, 
membercost * total_member_slots * total_member + guestcost * total_guest_slots * total_guest AS total_revenue
FROM Facilities AS F
LEFT JOIN (

SELECT facid, SUM(CASE WHEN memid !=0 THEN 1 ELSE 0 END ) total_member, 
SUM(CASE WHEN memid =0 THEN 1 ELSE 0 END ) total_guest, 
SUM(CASE WHEN memid !=0 THEN slots ELSE 0 END ) total_member_slots, 
SUM(CASE WHEN memid =0 THEN slots ELSE 0 END ) total_guest_slots
FROM Bookings
GROUP BY facid) AS B 
ON B.facid = F.facid
ORDER BY total_revenue
'''
cur.execute(sql_query).fetchall()

[('Table Tennis', 3240),
 ('Snooker Table', 5520),
 ('Pool Table', 14310),
 ('Badminton Court', 74353.5),
 ('Massage Room 2', 1183714.2),
 ('Tennis Court 2', 2335860),
 ('Tennis Court 1', 2381280),
 ('Squash Court', 3226510.0),
 ('Massage Room 1', 12337223.6)]

In [7]:
#Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order
sql_query =''' 
SELECT ( M1.surname ||' '|| M1.firstname ) AS member, ( M2.surname||' '|| M2.firstname ) AS recommendedby
FROM Members AS M1
INNER JOIN Members AS M2 ON M1.recommendedby = M2.memid
WHERE M1.surname != 'guest'
AND M2.surname != 'guest'
ORDER BY M1.surname, M1.firstname
'''
cur.execute(sql_query).fetchall()

[('Bader Florence', 'Stibbons Ponder'),
 ('Baker Anne', 'Stibbons Ponder'),
 ('Baker Timothy', 'Farrell Jemima'),
 ('Boothe Tim', 'Rownam Tim'),
 ('Butters Gerald', 'Smith Darren'),
 ('Coplin Joan', 'Baker Timothy'),
 ('Crumpet Erica', 'Smith Tracy'),
 ('Dare Nancy', 'Joplette Janice'),
 ('Genting Matthew', 'Butters Gerald'),
 ('Hunt John', 'Purview Millicent'),
 ('Jones David', 'Joplette Janice'),
 ('Jones Douglas', 'Jones David'),
 ('Joplette Janice', 'Smith Darren'),
 ('Mackenzie Anna', 'Smith Darren'),
 ('Owen Charles', 'Smith Darren'),
 ('Pinker David', 'Farrell Jemima'),
 ('Purview Millicent', 'Smith Tracy'),
 ('Rumney Henrietta', 'Genting Matthew'),
 ('Sarwin Ramnaresh', 'Bader Florence'),
 ('Smith Jack', 'Smith Darren'),
 ('Stibbons Ponder', 'Tracy Burton'),
 ('Worthington-Smyth Henry', 'Smith Tracy')]

In [8]:
# Q12: Find the facilities with their usage by member, but not guests 
sql_query ='''
SELECT F.name AS facility_name, 
SUM(CASE WHEN B.memid !=0 THEN 1 ELSE 0 END ) AS total_usage_by_member
FROM Facilities AS F
INNER JOIN Bookings AS B ON F.facid = B.facid
INNER JOIN Members AS M ON B.memid = M.memid
GROUP BY facility_name'''
cur.execute(sql_query).fetchall()

[('Badminton Court', 344),
 ('Massage Room 1', 421),
 ('Massage Room 2', 27),
 ('Pool Table', 783),
 ('Snooker Table', 421),
 ('Squash Court', 195),
 ('Table Tennis', 385),
 ('Tennis Court 1', 308),
 ('Tennis Court 2', 276)]

In [12]:
# Q13: Find the facilities usage by month, but not guests 
sql_query='''
SELECT F.name AS facility_name,
MONTH , total_member_usage
FROM Facilities AS F
INNER JOIN 
(SELECT memid, facid, SUM(CASE WHEN memid !=0 THEN 1 ELSE 0 END ) AS total_member_usage, 
(select strftime('%m', starttime)) AS month
FROM Bookings
GROUP BY facid,MONTH) AS B 
ON F.facid = B.facid
INNER JOIN Members AS M 
ON M.memid = B.memid'''
cur.execute(sql_query).fetchall()

[('Tennis Court 1', '07', 65),
 ('Tennis Court 1', '08', 111),
 ('Tennis Court 1', '09', 132),
 ('Tennis Court 2', '07', 41),
 ('Tennis Court 2', '08', 109),
 ('Tennis Court 2', '09', 126),
 ('Badminton Court', '07', 51),
 ('Badminton Court', '08', 132),
 ('Badminton Court', '09', 161),
 ('Table Tennis', '07', 48),
 ('Table Tennis', '08', 143),
 ('Table Tennis', '09', 194),
 ('Massage Room 1', '07', 77),
 ('Massage Room 1', '08', 153),
 ('Massage Room 1', '09', 191),
 ('Massage Room 2', '07', 4),
 ('Massage Room 2', '08', 9),
 ('Massage Room 2', '09', 14),
 ('Squash Court', '07', 23),
 ('Squash Court', '08', 85),
 ('Squash Court', '09', 87),
 ('Snooker Table', '07', 68),
 ('Snooker Table', '08', 154),
 ('Snooker Table', '09', 199),
 ('Pool Table', '07', 103),
 ('Pool Table', '08', 272),
 ('Pool Table', '09', 408)]